In [24]:
from newspaper import Article
from datasets import Dataset
import pandas as pd
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
HF_KEY = os.getenv("HF_KEY")
login(HF_KEY)

def scrape_articles_to_dataset(article_urls):
    data = {
        "title": [],
        "text": [],
    }
    
    for url in article_urls:
        try:
            # Create an Article object
            article = Article(url, language='ar')  # Assuming Urdu or Arabic
            article.download()
            article.parse()
            
            # Add title and text to data
            data["title"].append(article.title)
            data["text"].append(article.text)
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")
    
    # Create a Hugging Face dataset
    dataset = Dataset.from_dict(data)
    
    return dataset

def append_and_push_dataset(article_urls, hub_repo):
    try:
        existing_dataset = load_dataset(hub_repo)
        existing_df = pd.DataFrame(existing_dataset['train'])
    except Exception as e:
        print(f"Failed to load dataset from {hub_repo}: {e}")
        existing_df = pd.DataFrame(columns=["title", "text"])

    # Step 2: Scrape new articles
    new_dataset = scrape_articles_to_dataset(article_urls)
    
    # Convert the new dataset to a DataFrame for easier appending
    new_df = pd.DataFrame(new_dataset)

    # Step 3: Append the new rows to the existing dataset
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)

    # Step 4: Convert the updated DataFrame back to a Hugging Face Dataset
    updated_dataset = Dataset.from_pandas(updated_df)
    
    # Step 5: Push the updated dataset back to Hugging Face Hub
    updated_dataset.push_to_hub(hub_repo)

article_urls = [
    "https://jang.com.pk/news/1394720",
    "https://jang.com.pk/news/1394722",
    "https://jang.com.pk/news/1394784",
    "https://jang.com.pk/news/1394721",
]

hub_repo = "KANZOO/scrapped_articles"  # Replace with your Hugging Face repository

append_and_push_dataset(article_urls, hub_repo)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Kanza Nasim\.cache\huggingface\token
Login successful
Failed to load dataset from KANZOO/scrapped_articles: name 'load_dataset' is not defined


Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.36s/it]
